## Imports

In [1]:
import sys

sys.path.append("..")

In [2]:
from src.transformers import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Load Data

In [3]:
# Set default logging level
# Change to logging.INFO to see related output
logging.basicConfig(level=logging.INFO, force=True)

In [4]:
# Read data and set indices
train = pd.read_csv("../data/raw/train.csv")
test = pd.read_csv("../data/raw/test.csv")

train.set_index("listing_id", inplace=True)
train.drop_duplicates(inplace=True)

test.set_index("listing_id", inplace=True)
test.drop_duplicates(inplace=True)

## Construct Pipeline

In [7]:
data_pipeline = Pipeline(
    [
        ("pre_processing", PreProcessing()),
        (
            "imp_manufactured",
            GroupMissingValueImputer(
                "manufactured", ["make", "model", "type_of_vehicle"], "first"
            ),
        ),
        ("imp_power", GroupMissingValueImputer("power", ["type_of_vehicle"], "mean")),
        (
            "imp_engine_cap",
            GroupMissingValueImputer("engine_cap", ["make", "type_of_vehicle"], "mean"),
        ),
        (
            "imp_curb_weight",
            GroupMissingValueImputer("curb_weight", ["make", "type_of_vehicle"]),
        ),
        #         ("add_cat", SplitValuesToColumn("category")),
        # Columns 9 to 15
        (
            "curb_weight",
            CarSpecificationsTransformer(
                "curb_weight", ["make", "model", "type_of_vehicle", "manufactured"]
            ),
        ),
        (
            "power",
            CarSpecificationsTransformer(
                "power", ["make", "model", "type_of_vehicle", "manufactured"]
            ),
        ),
        (
            "fuel_type",
            CarSpecificationsTransformer("fuel_type", ["make", "model"], "mode"),
        ),
        (
            "engine_cap",
            CarSpecificationsTransformer(
                "engine_cap", ["make", "model", "type_of_vehicle", "manufactured"]
            ),
        ),
        ("fuel_type_missing", CarSpecsMissingWithTypeOfVehicle(["fuel_type"], "mode")),
        (
            "car_spec_missing",
            CarSpecsMissingWithTypeOfVehicle(
                ["curb_weight", "power", "engine_cap"], "mean"
            ),
        ),
        # Columns 15 and above
        ("coe_start_date", CoeStartDateFeatureCreator()),
        ("vehicle_age", AgeFeatureCreator()),
        (
            "omv",
            HierarchicalGroupImputer(
                "omv",
                [["make", "model", "vehicle_age"], ["make", "model"]],
                "mean",
                False,
            ),
        ),
        ("coe", CoeTransformer()),
        ("arf", ArfTransformer()),
        ("parf", ParfFeatureCreator()),
        ("coe_rebate", CoeRebateFeatureCreator()),
        ("dereg_value_computed", DeregValueComputedFeatureCreator()),
        ("dereg_value", DeregValueTransformer()),
        ("depreciation", DepreciationTransformer()),
        (
            "mileage",
            HierarchicalGroupImputer(
                "mileage",
                [["make", "model", "vehicle_age"], ["vehicle_age"]],
                "mean",
            ),
        ),
        ("opc_scheme", OpcSchemeTransformer()),
        # TODO: transform engine_cap as ranges as defined in https://www.sgcarmart.com/services/roadtax_calculator.php
        # This will further improve the accuracy of imputed road_tax values
        (
            "road_tax",
            HierarchicalGroupImputer(
                "road_tax",
                [
                    ["engine_cap", "opc_scheme", "vehicle_age", "fuel_type"],
                    ["engine_cap", "opc_scheme"],
                    ["opc_scheme"],
                ],
                "mean",
            ),
        ),
        ("lifespan_restriction", LifespanRestrictionFeatureCreator()),
        (
            "features_count",
            CountUniqueItemsFeatureCreator("features", "features_count"),
        ),
        (
            "accessories_count",
            CountUniqueItemsFeatureCreator("accessories", "accessories_count"),
        ),
    ]
)

In [9]:
trainX = train.loc[:, train.columns != "price"]
trainY = train.loc[:, "price"]
cleaned_train = data_pipeline.fit_transform(trainX)
cleaned_train.head()

/Users/krishnakumar/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
INFO:root:HierarchicalGroupImputer - found 41 null values to impute for omv
INFO:root:HierarchicalGroupImputer - 33 null values left to impute for omv
INFO:root:HierarchicalGroupImputer - 12 null values left to impute for omv
INFO:root:HierarchicalGroupImputer - Dropping 12 rows with null omv values
INFO:root:DepreciationTransformer - found 411 rows with null depreciation
INFO:root:HierarchicalGroupImputer - found 3500 null values to impute for mileage
INFO:root:HierarchicalGroupImputer - 531 null values left to impute for mileage
INFO:root:HierarchicalGroupImputer 

,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,...,coe_start_year,coe_expiry_months,vehicle_age,is_parf_car,parf,coe_rebate,dereg_value_computed,lifespan_restriction,features_count,accessories_count
listing_id,,,,,,,,,,,,,,,,,,,,,
365241,rover 216 cabriolet (coe till 04/2029),rover,216,new 10 years coe! affordable rare beautiful ca...,1993.0,NaN,1993-06-22,sports car,coe car,auto,...,2019,92.217765,28.0,0,0.0,20115.000000,20115.000000,1,4,0
397461,audi tt roadster 1.8a tfsi s-tronic (new 10-yr...,audi,tt,138000,2016.0,NaN,2013-01-09,sports car,"coe car, low mileage car",auto,...,2013,15.850017,8.0,0,0.0,8005.859045,8005.859045,1,2,6
540570,aston martin db9 cabriolet 6.0a (coe till 04/2...,aston martin,db9,new number plate! low mileage! very well maint...,2009.0,01-feb-2009,2011-01-27,sports car,"imported used vehicle, coe car, low mileage car",auto,...,2019,92.218802,12.0,0,0.0,27213.000000,27213.000000,1,4,5
592916,isuzu cyz52r (coe till 07/2027),isuzu,cyz52r,28 tons lorry with palfinger pk18500 crane.,2007.0,NaN,2007-11-12,truck,coe car,manual,...,2017,71.211036,14.0,0,0.0,20218.000000,20218.000000,-1,1,0
642522,mitsubishi fuso fighter fk61 (coe till 12/2029),mitsubishi,fuso,"very low mileage of 93,175 km done only. 6 ton...",2009.0,NaN,2009-12-04,truck,"coe car, low mileage car",manual,...,2019,99.351761,12.0,0,0.0,21176.000000,21176.000000,-1,0,0


In [11]:
trainX.shape, cleaned_train.shape

((16780, 31), (16126, 42))

In [8]:
# cleaned_test = data_pipeline.transform(test)
# cleaned_test.head()

## Write to disk

In [ ]:
# cleaned_train.to_csv("../data/processed/train.csv")
# cleaned_test.to_csv("../data/processed/train.csv")